In [8]:
from google.colab import drive
# Step 1: Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
import cv2
import numpy as np
from collections import defaultdict

# Define dataset paths
dataset_root = "/content/drive/MyDrive/TharuWorks/Dataset/train"  # Adjust path
images_dir = os.path.join(dataset_root, "images")  # RGB images
depth_labels_dir = os.path.join(dataset_root, "depth_labels")  # Labels with depth
tracked_labels_dir = os.path.join(dataset_root, "tracked_labels")  # New tracked labels
os.makedirs(tracked_labels_dir, exist_ok=True)

# Define relevant class IDs (pedestrians & vehicles)
VALID_CLASSES = {11, 22}  # Adjust according to dataset

# Step 1: Group frames by video
video_frames = defaultdict(list)

for label_file in os.listdir(depth_labels_dir):
    if label_file.endswith(".txt"):
        video_id = "_".join(label_file.split("_")[:2])  # Extract 'w5_44' as video ID
        video_frames[video_id].append(label_file)

# Step 2: Process each video separately
for video_id, frames in video_frames.items():
    frames.sort()  # Ensure frames are processed in order

    prev_tracked_objects = {}  # Dictionary to store tracked objects (previous frame)

    for frame_index, label_file in enumerate(frames):
        label_path = os.path.join(depth_labels_dir, label_file)
        output_label_path = os.path.join(tracked_labels_dir, label_file)  # Save updated labels

        # Read label file
        with open(label_path, "r") as f:
            lines = f.readlines()

        updated_lines = []
        tracked_objects = {}

        # Step 3: Find the closest valid object(s) in the frame
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 6:
                continue  # Skip invalid lines

            class_id, x_center, y_center, width, height, depth = map(float, parts)

            # Filter only pedestrians and vehicles
            if int(class_id) not in VALID_CLASSES:
                updated_lines.append(line)  # Keep unchanged
                continue

            object_id = (x_center, y_center, width, height, class_id)  # Include class_id in object tracking
            tracked_objects[object_id] = depth  # Store depth

        # Step 4: Track objects from previous frame
        if frame_index > 0:
            for obj_id, prev_depth in prev_tracked_objects.items():
                # If the object exists in the new frame, update its depth
                if obj_id in tracked_objects:
                    tracked_objects[obj_id] = min(prev_depth, tracked_objects[obj_id])  # Keep the closest distance
                else:
                    # Object disappeared, keep last known depth
                    tracked_objects[obj_id] = prev_depth

        # Step 5: Determine the closest object(s)
        sorted_objects = sorted(tracked_objects.items(), key=lambda x: x[1])  # Sort by depth (closest first)

        # Step 6: Mark the closest object(s) and calculate actual distance
        min_depth = sorted_objects[0][1] if sorted_objects else None  # Closest depth
        closest_objects = [sorted_objects[0]] if sorted_objects else []  # Start with closest object

        # If another object is within a 10% range of the closest, also track it
        for obj in sorted_objects[1:]:
            if obj[1] <= min_depth * 1.1:  # Within 10% depth range
                closest_objects.append(obj)

        # Step 7: Write updated labels
        for obj, depth in tracked_objects.items():
            x_center, y_center, width, height, class_id = obj  # Extract class_id
            is_closest = any(c[0] == obj for c in closest_objects)
            updated_line = f"{int(class_id)} {x_center} {y_center} {width} {height} {depth:.4f} {'closest' if is_closest else ''}\n"
            updated_lines.append(updated_line)

        # Save updated labels
        with open(output_label_path, "w") as f:
            f.writelines(updated_lines)

        # Update tracked objects for next frame
        prev_tracked_objects = tracked_objects.copy()

        print(f"✅ Processed {label_file} and saved updated tracking labels.")

print("🎯 Object tracking & distance marking complete!")


✅ Processed positive_w1-50_frame_0_jpg.rf.361f85e82a21d8da5931f383684dd9d6.txt and saved updated tracking labels.
✅ Processed positive_w1-50_frame_90_jpg.rf.c0734a9bfc40aeb49cb29055ac0de234.txt and saved updated tracking labels.
✅ Processed positive_w1-27_frame_0_jpg.rf.92a7b7b6e818910b3c8e577333870785.txt and saved updated tracking labels.
✅ Processed positive_w1-27_frame_180_jpg.rf.8664bc67a17314b91b8c88ebf9e1d0da.txt and saved updated tracking labels.
✅ Processed positive_w1-27_frame_90_jpg.rf.27403f5338e47a58cd4be2e73208c2e2.txt and saved updated tracking labels.
✅ Processed positive_w2_11_frame_0_jpg.rf.53538afcebc7a2ba6931bcf65e03bc3d.txt and saved updated tracking labels.
✅ Processed positive_w2_11_frame_180_jpg.rf.0aedcc363310f069f1f4a6753000575c.txt and saved updated tracking labels.
✅ Processed positive_w2_11_frame_270_jpg.rf.92969dd92ca9705dc37c7c9a460ab80f.txt and saved updated tracking labels.
✅ Processed positive_w2_11_frame_90_jpg.rf.c4d86a18237a68539d676557a2aa327a.txt 

In [5]:
import os
import cv2
import numpy as np
from collections import defaultdict

# Define dataset paths
dataset_root = "/content/drive/MyDrive/TharuWorks/Dataset/val"  # Adjust path
images_dir = os.path.join(dataset_root, "images")  # RGB images
depth_labels_dir = os.path.join(dataset_root, "depth_labels")  # Labels with depth
tracked_labels_dir = os.path.join(dataset_root, "tracked_labels")  # New tracked labels
os.makedirs(tracked_labels_dir, exist_ok=True)

# Define relevant class IDs (pedestrians & vehicles)
VALID_CLASSES = {11, 22}  # Adjust according to your dataset

# Step 1: Group frames by video
video_frames = defaultdict(list)

for label_file in os.listdir(depth_labels_dir):
    if label_file.endswith(".txt"):
        video_id = "_".join(label_file.split("_")[:2])  # Extract 'w5_44' as video ID
        video_frames[video_id].append(label_file)

# Step 2: Process each video separately
for video_id, frames in video_frames.items():
    frames.sort()  # Ensure frames are processed in order

    prev_tracked_objects = {}  # Dictionary to store tracked objects

    for frame_index, label_file in enumerate(frames):
        label_path = os.path.join(depth_labels_dir, label_file)
        output_label_path = os.path.join(tracked_labels_dir, label_file)  # Save updated labels

        # Read label file
        with open(label_path, "r") as f:
            lines = f.readlines()

        updated_lines = []
        tracked_objects = {}

        # Step 3: Find the closest valid object(s) in the frame
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 6:
                continue  # Skip invalid lines

            class_id, x_center, y_center, width, height, depth = map(float, parts)

            # Filter only pedestrians and vehicles
            if int(class_id) not in VALID_CLASSES:
                updated_lines.append(line)  # Keep unchanged
                continue

            object_id = (x_center, y_center, width, height, class_id)  # Include class_id in object tracking
            tracked_objects[object_id] = depth  # Store depth

        # Step 4: Track objects from previous frame
        if frame_index > 0:
            for obj_id, prev_depth in prev_tracked_objects.items():
                # If the object exists in the new frame, update its depth
                if obj_id in tracked_objects:
                    tracked_objects[obj_id] = min(prev_depth, tracked_objects[obj_id])  # Keep the closest distance
                else:
                    # Object disappeared, keep last known depth
                    tracked_objects[obj_id] = prev_depth

        # Step 5: Write updated labels
        for obj_id, depth in tracked_objects.items():
            x_center, y_center, width, height, class_id = obj_id  # Extract class_id
            updated_line = f"{int(class_id)} {x_center} {y_center} {width} {height} {depth:.4f}\n"
            updated_lines.append(updated_line)

        # Save updated labels
        with open(output_label_path, "w") as f:
            f.writelines(updated_lines)

        # Update tracked objects for next frame
        prev_tracked_objects = tracked_objects.copy()

        print(f"✅ Processed {label_file} and saved updated tracking labels.")

print("🎯 Object tracking & distance marking complete!")


✅ Processed negative_w10_2_frame_0_jpg.rf.42043daa58b3312bffd8cab0bbe663ec.txt and saved updated tracking labels.
✅ Processed negative_w10_2_frame_180_jpg.rf.7ff3742db7a30048cd92927fe2b0728c.txt and saved updated tracking labels.
✅ Processed negative_w10_2_frame_90_jpg.rf.d1b2aec1cecc98ac228c201d7fe568de.txt and saved updated tracking labels.
✅ Processed negative_w10_30_frame_0_jpg.rf.dc08c5d4cf80f812ee7f40e4e70ca051.txt and saved updated tracking labels.
✅ Processed negative_w10_30_frame_180_jpg.rf.03f9deb5cf33c91c91da877b082913a3.txt and saved updated tracking labels.
✅ Processed negative_w10_30_frame_90_jpg.rf.da25d9eebba985ddea66810b021e74ec.txt and saved updated tracking labels.
✅ Processed negative_w10_41_frame_0_jpg.rf.643af631b91c611776bb4143b2f9c93d.txt and saved updated tracking labels.
✅ Processed negative_w10_41_frame_180_jpg.rf.9eb9b66a73224936b970fe21d7c8b137.txt and saved updated tracking labels.
✅ Processed negative_w10_41_frame_90_jpg.rf.071ac4bba91429def4452bc5c3b2e68